In [6]:
from os import listdir
import pandas as pd
import numpy as np
from os.path import isfile, join
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import os
import seaborn as sns
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import nltk
from nltk.stem import WordNetLemmatizer 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
nltk.download('wordnet')

my_path = os.getcwd()
os.chdir(my_path)
file = 'training-Obama-Romney-tweets.xlsx'


obama = 'Obama'
romney = 'Romney'

tweets_df = pd.read_excel(file, sheet_name = obama)
# dfs = {sheet_name: tweets.parse(sheet_name) 
#           for sheet_name in tweets.sheet_names}

tweets_df.dropna(subset = ['Unnamed: 4'], inplace=True)
tweets_df['Unnamed: 4']=tweets_df["Unnamed: 4"].map(str)


tweets_df = tweets_df.drop(tweets_df[tweets_df['Unnamed: 4']=='!!!!'].index,axis=0)
tweets_df = tweets_df.drop(tweets_df[tweets_df['Unnamed: 4']=='irrevelant'].index,axis=0)
tweets_df = tweets_df.drop(tweets_df[tweets_df['Unnamed: 4']=='irrelevant'].index,axis=0)
tweets_df = tweets_df.drop(tweets_df[tweets_df['Unnamed: 4']=='2'].index,axis=0)
tweets_df = tweets_df.drop(0,axis=0)
tweets_df['Unnamed: 4']=tweets_df["Unnamed: 4"].map(int)
tweets_df['Anootated tweet'] = tweets_df['Anootated tweet'].str.replace(r'<[^<>]*>', '', regex=True)

X=tweets_df['Anootated tweet']
Y=tweets_df['Unnamed: 4']

l1 = []
for i in range(len(Y)):
    if Y.iloc[i] not in l1:
        l1.append(Y.iloc[i])
print("y is :",l1)

tweets_df['Unnamed: 4'].unique()

# # Plotting the distribution for dataset.
# ax = tweets_df.groupby('Unnamed: 4').count().plot(kind='bar', title='Distribution of data',legend=False)
# ax.set_xticklabels(['Neutral','Positive','Negative'], rotation=0)
# # Storing data in lists.
# text, sentiment = list(tweets_df['Anootated tweet']), list(tweets_df['Unnamed: 4'])

# sns.countplot(x='Unnamed: 4', data=tweets_df)



data=tweets_df[['Anootated tweet','Unnamed: 4']]
data['Unnamed: 4'].unique()

data_pos = data[data['Unnamed: 4'] == 1]
data_neg = data[data['Unnamed: 4'] == -1]
data_neu = data[data['Unnamed: 4'] == 0]

dataset = pd.concat([data_pos, data_neg, data_neu])

dataset['Anootated tweet']=dataset['Anootated tweet'].str.lower()
# dataset['Anootated tweet'].tail()

stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

STOPWORDS = list(stopwordlist)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
dataset['Anootated tweet'] = dataset['Anootated tweet'].apply(lambda text: cleaning_stopwords(text))
dataset['Anootated tweet'].head()

english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
dataset['Anootated tweet']= dataset['Anootated tweet'].apply(lambda x: cleaning_punctuations(x))
dataset['Anootated tweet'].tail()

def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)
dataset['Anootated tweet'] = dataset['Anootated tweet'].apply(lambda x: cleaning_repeating_char(x))
dataset['Anootated tweet'].tail()

def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
dataset['Anootated tweet'] = dataset['Anootated tweet'].apply(lambda x: cleaning_URLs(x))
dataset['Anootated tweet'].tail()

def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['Anootated tweet'] = dataset['Anootated tweet'].apply(lambda x: cleaning_numbers(x))
dataset['Anootated tweet'].tail()

tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
dataset['Anootated tweet'] = dataset['Anootated tweet'].apply(tokenizer.tokenize)


st = PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return text

dataset['Anootated tweet']= dataset['Anootated tweet'].apply(lambda x: stemming_on_text(x))
dataset['Anootated tweet']

lm = WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return text
dataset['Anootated tweet'] = dataset['Anootated tweet'].apply(lambda x: lemmatizer_on_text(x))
dataset['Anootated tweet']

X = dataset['Anootated tweet']

cleaned_data = []
for i in range(len(X)):
    tweet = re.sub('[^a-zA-Z]',' ',str(X.iloc[i]))
    tweet = tweet.split()
    tweet = [st.stem(word) for word in tweet]
    tweet = ' '.join(tweet)
    cleaned_data.append(tweet)

vectoriser = TfidfVectorizer(ngram_range=(1,2),stop_words=['obama','barack','romney','mitt'])


y = dataset['Unnamed: 4']

X_fin=vectoriser.fit_transform(cleaned_data).toarray()
y = dataset['Unnamed: 4']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=26105111)

# X_train.shape
# y_train.shape


BNBmodel = BernoulliNB()
BNBmodel.fit(X_train, y_train)
model_Evaluate(BNBmodel)
y_pred1 = BNBmodel.predict(X_test)

cf=classification_report(y_test,y_pred)



[nltk_data] Downloading package wordnet to /Users/aikyab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


y is : [0, 1, -1]


ValueError: setting an array element with a sequence.